In [1]:
#select * from AINDEXMEMBERS
#where S_INFO_WINDCODE = '000300.SH' and YEAR(OPDATE)='2016' and MONTH(OPDATE)='12' and DAY(OPDATE)='09'

#注意FIND_IN_SET中的字符串中，代码间不能有空格，否则只有第一个代码有效
#另外尽量不要用OPDATE作为时间控制字段
#select * from ASHAREINTENSITYTREND
#where YEAR(OPDATE)='2016' and MONTH(OPDATE)='12' and DAY(OPDATE)='09'
#and FIND_IN_SET(S_INFO_WINDCODE,'603099.SH,002302.SZ')

#select * from ASHARECALENDAR where FIND_IN_SET(S_INFO_EXCHMARKET,'SZSE,SSE') and TRADE_DAYS BETWEEN 20110407 
#and 20110430 ORDER BY TRADE_DAYS desc limit 2
#select * from ASHARECALENDAR where FIND_IN_SET(S_INFO_EXCHMARKET,'SZSE,SSE') and TRADE_DAYS LIKE '202005%' ORDER BY TRADE_DAYS desc limit 2
#select TRADE_DAYS from ASHARECALENDAR where FIND_IN_SET(S_INFO_EXCHMARKET,'SSE') and TRADE_DAYS LIKE '202005%' ORDER BY TRADE_DAYS desc limit 1

In [2]:
import numpy as np
import pandas as pd
import rqdatac as rq
import calendar
import pymysql
import os
import re
import sqlalchemy
from sqlalchemy import create_engine

from rqdatac import *
from datetime import datetime, timedelta, date
from collections import OrderedDict
from params import connect_info

work_dir = os.path.dirname(os.path.abspath('__file__'))
file_path = os.path.join(work_dir, 'factor_preprocess', 'factors')
save_path = os.path.join(work_dir, 'factor_preprocess', 'factors_done')
print(file_path)

engine = create_engine(connect_info)

global file_path, save_path, industry_benchmark 

/AI2/IE/factor_preprocess/factors


In [3]:
dates = ["2019-1-1", date.today().strftime('%Y-%m-%d')]
start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
dates_dict = OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys()
dates_series = [_ for _ in dates_dict]
#dates_series

In [4]:
for fpath in os.listdir(file_path)[:]:
    print(fpath)
    all_data = pd.DataFrame()
    date_range = fpath.split('.')[0].split('-')
    date_range = date_range[0] + date_range[1] + '%%'
    last_day_sql_cmd = f"select TRADE_DAYS from ASHARECALENDAR where FIND_IN_SET(S_INFO_EXCHMARKET,'SSE') and \
                        TRADE_DAYS LIKE '{date_range}' ORDER BY TRADE_DAYS desc limit 1"
    trade_day = pd.read_sql(sql= last_day_sql_cmd, con = engine).TRADE_DAYS.tolist()[0]
    #print(trade_day[:4],trade_day[4:6],trade_day[6:8])

    origin_df = pd.read_csv(os.path.join(file_path, fpath), engine='c',encoding='gbk', index_col=[0])
    #print(origin_df.iloc[:,:8])
    sec_code = str(origin_df.code.tolist()[:])
    origin_df = origin_df.iloc[:,:7].set_index(origin_df.code)
                    
    sec_code = re.sub('[\ \[\]\']', '', sec_code)
    sec_code = f"'{sec_code}'"
    sec_code_series = f'FIND_IN_SET(S_INFO_WINDCODE,{sec_code})'
    print(f'dealing with {fpath}')
    
    # round 1
    # 统计因子
    Indicator_Name = 'S_INFO_WINDCODE PCT_CHANGE_M ALPHA_DAY_1Y TURNOVER_M STD_DEVIATION_24M ' + \
                    'TURNOVER_D_FLOAT TURNOVER_W_FLOAT TURNOVER_M_FLOAT'
    Indicator_Name = Indicator_Name.replace(' ', ',')
    #Indicator_Name = '*'
    Table_Name = 'ASHAREYIELD'
    condiction = f" WHERE YEAR(TRADE_DT)={trade_day[0:4]} AND \
                    MONTH(TRADE_DT)={trade_day[4:6]} AND \
                    DAY(TRADE_DT)={trade_day[6:8]} AND \
                    {sec_code_series}" 
    sql_cmd = f'select {Indicator_Name} from {Table_Name} {condiction}'
    #print(sql_cmd)
    
    print(Table_Name)
    sql_df = pd.read_sql(sql= sql_cmd, con = engine)
    sql_df = sql_df.set_index(sql_df.S_INFO_WINDCODE)
    sql_df.rename(columns={'PCT_CHANGE_M': 'PCT_CHG_NM'}, inplace=True)
    sql_df.rename(columns={'TURNOVER_D_FLOAT': 'STOA_Barra'}, inplace=True)
    sql_df.rename(columns={'TURNOVER_W_FLOAT': 'STOW_Barra'}, inplace=True)
    sql_df.rename(columns={'TURNOVER_M_FLOAT': 'STOM_Barra'}, inplace=True)
    #print(origin_df.shape,sql_df.shape)
    all_data = pd.concat([origin_df, sql_df], axis = 1)
    
    # round 2
    # 技术因子
    Indicator_Name = 'S_INFO_WINDCODE MA_20D MACD_DIFF MACD_DEA MACD_MACD'
    Indicator_Name = Indicator_Name.replace(' ', ',')
    #Indicator_Name = '*'
    Table_Name = 'ASHAREINTENSITYTREND'
    condiction = f" WHERE YEAR(TRADE_DT)={trade_day[0:4]} AND \
                    MONTH(TRADE_DT)={trade_day[4:6]} AND \
                    DAY(TRADE_DT)={trade_day[6:8]} AND \
                    {sec_code_series}" 
    sql_cmd = f'select {Indicator_Name} from {Table_Name} {condiction}'
    #print(sql_cmd)
            
    print(Table_Name)
    sql_df = pd.read_sql(sql= sql_cmd, con = engine)
    sql_df = sql_df.set_index(sql_df.S_INFO_WINDCODE)
    #print(origin_df.shape,sql_df.shape)
    all_data = pd.concat([all_data, sql_df], axis = 1)
    
    # round 3
    # 市值相关
    Indicator_Name = 'S_INFO_WINDCODE S_VAL_MV S_VAL_PE S_VAL_PE_TTM S_DQ_MV S_VAL_PCF_NCFTTM '+ \
                    'NET_CASH_FLOWS_OPER_ACT_TTM NET_INCR_CASH_CASH_EQU_TTM S_VAL_PB_NEW NET_PROFIT_PARENT_COMP_TTM'
    Indicator_Name = Indicator_Name.replace(' ', ',')
    Table_Name = 'ASHAREEODDERIVATIVEINDICATOR'
    condiction = f" WHERE YEAR(TRADE_DT)={trade_day[0:4]} AND \
                    MONTH(TRADE_DT)={trade_day[4:6]} AND \
                    DAY(TRADE_DT)={trade_day[6:8]} AND \
                    {sec_code_series}" 
    sql_cmd = f'select {Indicator_Name} from {Table_Name} {condiction}'
    #print(sql_cmd)
            
    print(Table_Name)
    sql_df = pd.read_sql(sql= sql_cmd, con = engine)
    sql_df = sql_df.set_index(sql_df.S_INFO_WINDCODE)
    sql_df['LNCAP_barra'] = np.log(sql_df.S_DQ_MV)
    sql_df['BP'] = (1 / sql_df.S_DQ_MV)
    sql_df['BTOP_Barra'] = (1 / sql_df.S_DQ_MV)
    sql_df['CETOP_Barra'] = (1 / sql_df.S_VAL_PCF_NCFTTM)
    sql_df['ETOP_Barra'] = (1 / sql_df.S_VAL_PE)
    sql_df['EP'] = (1 / sql_df.S_VAL_PE_TTM)
    sql_df['GPE'] = (sql_df.NET_PROFIT_PARENT_COMP_TTM / sql_df.S_VAL_PE_TTM)
    sql_df['OCFP'] = (1 / sql_df.NET_CASH_FLOWS_OPER_ACT_TTM)
    sql_df['NCFP'] = (1 / sql_df.NET_INCR_CASH_CASH_EQU_TTM)
    sql_df.rename(columns={'S_DQ_MV': 'MKT_CAP_FLOAT'}, inplace=True)
    #print(origin_df.shape,sql_df.shape)
    all_data = pd.concat([all_data, sql_df], axis = 1)
    
    # round 4
    # 交易状态
    Indicator_Name = 'S_INFO_WINDCODE S_DQ_TRADESTATUS'
    Indicator_Name = Indicator_Name.replace(' ', ',')
    Table_Name = 'ASHAREEODPRICES'
    condiction = f" WHERE YEAR(TRADE_DT)={trade_day[0:4]} AND \
                    MONTH(TRADE_DT)={trade_day[4:6]} AND \
                    DAY(TRADE_DT)={trade_day[6:8]} AND \
                    {sec_code_series}" 
    sql_cmd = f'select {Indicator_Name} from {Table_Name} {condiction}'
    #print(sql_cmd)
            
    print(Table_Name)
    sql_df = pd.read_sql(sql= sql_cmd, con = engine)
    sql_df = sql_df.set_index(sql_df.S_INFO_WINDCODE)
    #print(origin_df.shape,sql_df.shape)
    all_data = pd.concat([all_data, sql_df], axis = 1)
    
    # round 5
    # 技术相关 反转
    Indicator_Name = 'S_INFO_WINDCODE BIAS'
    Indicator_Name = Indicator_Name.replace(' ', ',')
    Table_Name = 'ASHARESWINGREVERSETREND'
    condiction = f" WHERE YEAR(TRADE_DT)={trade_day[0:4]} AND \
                    MONTH(TRADE_DT)={trade_day[4:6]} AND \
                    DAY(TRADE_DT)={trade_day[6:8]} AND \
                    {sec_code_series}" 
    sql_cmd = f'select {Indicator_Name} from {Table_Name} {condiction}'
    #print(sql_cmd)
            
    print(Table_Name)
    sql_df = pd.read_sql(sql= sql_cmd, con = engine)
    sql_df = sql_df.set_index(sql_df.S_INFO_WINDCODE)
    #print(origin_df.shape,sql_df.shape)
    all_data = pd.concat([all_data, sql_df], axis = 1)
    
    # round 6
    # 财务因子
    
    
    all_data = all_data.reset_index(drop = True)
    all_data['is_open1'] = all_data.S_DQ_TRADESTATUS.apply(lambda x:'TRUE' if x == '交易' else 'FALSE')
    all_data = all_data.dropna(subset=['S_DQ_TRADESTATUS'])
    del all_data['S_INFO_WINDCODE']
    del all_data['S_DQ_TRADESTATUS']
    
    all_data = all_data.dropna()
    all_data['No'] = all_data.index
    all_data = all_data.set_index(all_data.No, drop = True)
    
    all_data.to_csv(save_path, encoding='gbk')

222222


IndexError: list index out of range